In [237]:
import requests
import pandas as pd
import logging
from datetime import datetime, timedelta, timezone, date
from OddsJamClient import OddsJamClient


In [173]:
api_key_path = 'api_key.txt'

def load_api_key(path):
    with open(path, 'r') as file:
        return file.readline().strip()

api_key = load_api_key(api_key_path)


In [174]:
def fetch_sports_markets(sport, league, api_key, sportsbook=['Pinnacle']):
    url = "https://api.opticodds.com/api/v3/markets"
    params = {
        'sport': sport,
        'league': league,
        'key':api_key
    }
    
    # Optional: Specify a sportsbook if required
    if sportsbook:
        params['sportsbook'] = sportsbook

    headers = {'Authorization': f'Bearer {api_key}'}
    
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # Assuming the data is formatted in a list of markets, convert it into a DataFrame
        markets_df = pd.DataFrame(data['data'])
        return markets_df
    else:
        print(f"Failed to fetch markets: {response.status_code} - {response.text}")
        return pd.DataFrame()
def american_to_decimal(odds):
    if odds > 0:
        return (odds / 100) + 1
    else:
        return (100 / abs(odds)) + 1

def implied_probability(decimal_odds):
    return 1 / decimal_odds

def remove_vig(implied_prob_a, implied_prob_b):
    no_vig_prob_a = implied_prob_a / (implied_prob_a + implied_prob_b)
    return no_vig_prob_a
def adjust_for_vig(implied_prob, vig_reduction=0.02):
    return implied_prob / (1 + vig_reduction)

def calculate_ev(true_prob, odds, stake=100):
    # Convert American odds to decimal odds
    decimal_odds = american_to_decimal(odds)
    
    # Calculate profit if win
    profit_if_win = (decimal_odds - 1) * stake
    
    # Calculate EV using the formula (True Probability * Profit if Win) - (Loss Probability * Stake)
    loss_prob = 1 - true_prob
    ev = (true_prob * profit_if_win) - (loss_prob * stake)
    
    return ev

def power_method_de_vig(prob, power=0.98):  # Use a more moderate power value
    return prob ** power

def find_plus_ev_bets(df, thresh):
    caesars_df = df[df['Sportsbook'] == 'Caesars']
    pinnacle_df = df[df['Sportsbook'] == 'Pinnacle']
    betmgm_df = df[df['Sportsbook'] == 'BetMGM']

    betmgm_df = betmgm_df.rename(columns={'Odds': 'Odds_betmgm', 'line': 'line_betmgm'})

    merged_df = pd.merge(caesars_df, pinnacle_df, on=['Game ID', 'Bet Name', 'Market Name'], suffixes=('_caesars', '_pinnacle'))

    merged_df = pd.merge(merged_df, betmgm_df, on=['Game ID', 'Bet Name', 'Market Name'], how='left')

    
    merged_df['decimal_odds_caesars'] = merged_df['Odds_caesars'].apply(american_to_decimal)
    merged_df['decimal_odds_pinnacle'] = merged_df['Odds_pinnacle'].apply(american_to_decimal)
    merged_df['decimal_odds_betmgm'] = merged_df['Odds_betmgm'].apply(american_to_decimal)
    
    merged_df['implied_prob_caesars'] = merged_df['decimal_odds_caesars'].apply(implied_probability)
    merged_df['implied_prob_pinnacle'] = merged_df['decimal_odds_pinnacle'].apply(implied_probability)
    merged_df['implied_prob_betmgm'] = merged_df['decimal_odds_betmgm'].apply(implied_probability)

    merged_df['true_prob_pinnacle'] = merged_df['implied_prob_pinnacle'].apply(lambda prob: adjust_for_vig(prob, 0.02))  # Assuming 2% is the vig you want to remove

    
    merged_df['EV_caesars'] = merged_df.apply(
        lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_caesars']) 
        if row['true_prob_pinnacle'] > row['implied_prob_caesars']  
        else -1, 
        axis=1)

    merged_df['EV_betmgm'] = merged_df.apply(
        lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_betmgm']) 
        if row['true_prob_pinnacle'] > row['implied_prob_betmgm']  
        else -1, 
        axis=1)

    positive_ev_bets = merged_df[(merged_df['EV_caesars'] > thresh) | (merged_df['EV_betmgm'] > thresh)]
    
    return positive_ev_bets

def fetch_game_data(game_ids, markets, sportsbooks, include_player_name=True):
    URL = "https://api-external.oddsjam.com/api/v2/game-odds"
    API_KEY = api_key
    all_data = []

    for chunk in [game_ids[i:i + 5] for i in range(0, len(game_ids), 5)]:
        for sportsbook in sportsbooks:
            params = {
                'key': API_KEY,
                'sportsbook': sportsbook,
                'game_id': chunk,
                'market_name': markets
            }
            response = requests.get(URL, params=params)
            if response.status_code == 200:
                data = response.json().get('data', [])
                all_data.extend(data)
            else:
                logging.error("Error fetching data: %s - %s", response.status_code, response.text)
    rows = []
    for game_data in all_data:
        home_team = game_data.get('home_team', 'Unknown')
        away_team = game_data.get('away_team', 'Unknown')
        odds_list = game_data.get('odds', [])
            
        for item in odds_list:
            market_name = item.get('market_name', '')
            sportsbook_name = item.get('sports_book_name', sportsbook)
            if market_name not in markets:
                continue

            row = {
                'Game ID': game_data.get('id', 'Unknown'),
                "Game Name": home_team+" vs "+away_team,
                "Bet Name":item.get('name',None),
                'Market Name': market_name,
                'Sportsbook': sportsbook_name,
                'line':item.get('bet_points',None),
                'Odds': item.get('price', None),
            }

            if include_player_name:
                player_name = item.get('selection', 'Unknown')
                row['Player Name'] = player_name

            rows.append(row)

    return pd.DataFrame(rows)


### MLB

In [175]:
def get_todays_game_ids():
    
    Client = OddsJamClient(api_key)
    Client.UseV2()
    
    # Get games for the league
    GamesResponse = Client.GetGames(league='mlb')
    Games = GamesResponse.Games
    
    # Filter games based on today's date
    games_data = [{'game_id': game.id, 'start_date': game.start_date} for game in Games]
    games_df = pd.DataFrame(games_data)
    games_df['start_date'] = pd.to_datetime(games_df['start_date'])
    #today = datetime.now().date()
    #todays_games = games_df[games_df['start_date'].dt.date == today]
    
    return games_df['game_id'].tolist()

    


def fetch_player_props(game_ids):
    player_markets = [
        "Player Hits Allowed", "Player Strikeouts", "Player Home Runs",
        "Player Bases", "Player Home Runs Yes/No", "Player Outs", "Player Earned Runs"
    ]
    return fetch_game_data(game_ids, player_markets, ['Pinnacle','BetMGM','Caesars'], include_player_name=True)

def fetch_game_props(game_ids):
    game_markets = [
        "Team Total", "1st Half Team Total", "1st Half Moneyline", "Moneyline",
        "No Runs First Inning", "1st Inning Moneyline 3-Way", "1st Half Run Line",
        "Total Runs", "1st Inning Total Runs", "1st Half Total Runs", "Run Line",
        "1st Inning Run Line", "Total Hits + Runs + Errors"
    ]
    return fetch_game_data(game_ids, game_markets, ['Pinnacle','BetMGM','Caesars'], include_player_name=False)


In [223]:
todays_ids = get_todays_game_ids()
pdf = fetch_player_props(game_ids=todays_ids)
gdf = fetch_game_props(todays_ids)

In [224]:
evdf = find_plus_ev_bets(pdf,thresh=5)
game_evdf = find_plus_ev_bets(gdf,thresh=5)
game_evdf['League'] = 'MLB'
evdf['League'] = 'MLB'
betmgmdf = evdf[['Game ID','League','Bet Name',"Game Name",'Market Name','line_betmgm','Odds_betmgm','line_pinnacle','Odds_pinnacle','EV_betmgm','true_prob_pinnacle']]
betmgmdf = betmgmdf.sort_values('EV_betmgm',ascending=False)
caesarsdf_mlb_player = evdf[['Game ID','League','Bet Name',"Game Name",'Market Name','line_caesars','Odds_caesars','line_pinnacle','Odds_pinnacle','EV_caesars','true_prob_pinnacle']]
caesars_gdf = game_evdf[['Game ID','League','Bet Name',"Game Name",'Market Name','line_caesars','Odds_caesars','line_pinnacle','Odds_pinnacle','EV_caesars','true_prob_pinnacle']]

In [225]:
pd.set_option('display.max_rows',100)

In [226]:
caesars_gdf = caesars_gdf.sort_values('EV_caesars',ascending=False)

In [227]:
#caesarsdf.to_csv('caesars_mlb_plays.csv',index=False)

In [228]:
caesarsdf_mlb_player

,Game ID,League,Bet Name,Game Name,Market Name,line_caesars,Odds_caesars,line_pinnacle,Odds_pinnacle,EV_caesars,true_prob_pinnacle
122,16196-34048-2024-09-22-13,MLB,Antonio Senzatela Over 4.5,Los Angeles Dodgers vs Colorado Rockies,Player Hits Allowed,4.5,-151.0,4.5,-187.0,6.183323,0.638792
149,37569-25683-2024-09-22-13,MLB,Andrew Vaughn Over 0.5,San Diego Padres vs Chicago White Sox,Player Home Runs,0.5,550.0,0.5,556.0,-1.000000,0.149450
203,37569-25683-2024-09-22-13,MLB,Andrew Vaughn Yes,San Diego Padres vs Chicago White Sox,Player Home Runs Yes/No,NaN,550.0,NaN,556.0,-1.000000,0.149450
226,54558-76765-2024-09-22-13,MLB,Aaron Judge Over 0.5,Oakland Athletics vs New York Yankees,Player Home Runs,0.5,170.0,0.5,178.0,-1.000000,0.352659
229,54558-76765-2024-09-22-13,MLB,Austin Wells Over 0.5,Oakland Athletics vs New York Yankees,Player Home Runs,0.5,360.0,0.5,327.0,5.616017,0.229600
231,54558-76765-2024-09-22-13,MLB,Giancarlo Stanton Over 0.5,NaN,Player Home Runs,0.5,310.0,0.5,256.0,12.910333,0.275391
282,54558-76765-2024-09-22-13,MLB,Aaron Judge Yes,Oakland Athletics vs New York Yankees,Player Home Runs Yes/No,NaN,170.0,NaN,178.0,-1.000000,0.352659
285,54558-76765-2024-09-22-13,MLB,Austin Wells Yes,Oakland Athletics vs New York Yankees,Player Home Runs Yes/No,NaN,360.0,NaN,327.0,5.616017,0.229600
287,54558-76765-2024-09-22-13,MLB,Giancarlo Stanton Yes,NaN,Player Home Runs Yes/No,NaN,310.0,NaN,256.0,12.910333,0.275391
299,54558-76765-2024-09-22-13,MLB,Joey Estes Under 3.5,Oakland Athletics vs New York Yankees,Player Earned Runs,3.5,-154.0,3.5,-199.0,7.620427,0.652502


### NCAAF

In [127]:
## NCAA Football
def get_ncaa_game_ids():
    # Endpoint
    endpoint = "https://api-external.oddsjam.com/api/v2/games"

    # Current week date range
    today = datetime.now(timezone.utc)
    start_date = today.isoformat()


    # Calculate the upcoming Monday or today if it's Monday
    days_until_monday = (7 - today.weekday()) % 7
    monday = today + timedelta(days=days_until_monday)
    
    # Set time to 11:59 pm and adjust for CST (UTC-6)
    end_date_cst = (monday + timedelta(days=1)).replace(hour=5, minute=59, second=59, microsecond=0, tzinfo=timezone.utc)
    # Parameters
    params = {
        "key": api_key,
        "sport": "football",
        "league": "NCAAF",
        "start_date_after": start_date,
        "start_date_before": end_date_cst.isoformat()
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check response
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None

    data = response.json()
    game_ids = [game['id'] for game in data['data']]
    return game_ids


In [164]:
# Example usage
api_key = api_key
ncaaf_markets = fetch_sports_markets('football', 'NCAAF', api_key)
ncaa_ids = get_ncaa_game_ids()
ncaaf_markets = ncaaf_markets['name'].tolist()
ncaaf_df = fetch_game_data(ncaa_ids,ncaaf_markets,['Pinnacle','BetMGM','Caesars'])
ncaaf_df_ev = find_plus_ev_bets(ncaaf_df,thresh=5)
ncaaf_df_ev['League'] = 'NCAAF'
betmgmdf_ncaa = ncaaf_df_ev[['Game ID','League','Bet Name',"Game Name",'Market Name','line_betmgm','Odds_betmgm','line_pinnacle','Odds_pinnacle','EV_betmgm','true_prob_pinnacle']]
betmgmdf_ncaa = betmgmdf_ncaa.sort_values('EV_betmgm',ascending=False)


In [165]:
caesars_ncaa = ncaaf_df_ev[['Game ID','League','Bet Name',"Game Name",'Market Name','line_caesars','Odds_caesars','line_pinnacle','Odds_pinnacle','EV_caesars','true_prob_pinnacle']]
caesars_ncaa = caesars_ncaa.sort_values('EV_caesars',ascending=False)


In [166]:
caesars_ncaa

,Game ID,League,Bet Name,Game Name,Market Name,line_caesars,Odds_caesars,line_pinnacle,Odds_pinnacle,EV_caesars,true_prob_pinnacle
308,99638-12398-24-38,NCAAF,Florida A&M,NaN,Moneyline,NaN,950.0,NaN,784.0,16.449295,0.110904
222,28505-12457-24-38,NCAAF,Arkansas Pine Bluff +34,NaN,Point Spread,34.0,-110.0,34.0,-133.0,6.837116,0.559623
238,36045-43311-24-38,NCAAF,North Carolina A&T +6,NaN,Point Spread,6.0,-110.0,6.0,-133.0,6.837116,0.559623
287,72783-44096-24-38,NCAAF,Under 60,NaN,Total Points,60.0,-110.0,60.0,-133.0,6.837116,0.559623
348,42719-22926-24-38,NCAAF,Furman +9,NaN,Point Spread,9.0,-105.0,9.0,-124.0,5.959050,0.542717
11,96808-18998-24-38,NCAAF,Under 48.5,NaN,Total Points,48.5,-110.0,48.5,-130.0,5.789351,0.554135
1,96808-18998-24-38,NCAAF,Northern Iowa +9.5,NaN,Point Spread,9.5,100.0,9.5,-116.0,5.301380,0.526507
326,21999-22078-24-38,NCAAF,Over 56,NaN,Total Points,56.0,-110.0,56.0,-128.0,5.075523,0.550396
336,29856-33023-24-38,NCAAF,Idaho State +3.5,Idaho State vs Southern Utah,Point Spread,3.5,-120.0,3.5,-130.0,1.591361,0.554135


In [130]:
#caesars_ncaa.to_csv('caesars_ncaaf_game_bets_9_20.csv',index=False,header=True)

### Soccer

In [219]:
"""
def get_soccer_game_ids(league):
    # Endpoint
    endpoint = "https://api-external.oddsjam.com/api/v2/games"

    # Current week date range
    today = datetime.now(timezone.utc)
    start_date = today.isoformat()


    # Calculate the upcoming Monday or today if it's Monday
    days_until_monday = (7 - today.weekday()) % 7
    monday = today + timedelta(days=days_until_monday)
    
    # Set time to 11:59 pm and adjust for CST (UTC-6)
    end_date_cst = (monday + timedelta(days=1)).replace(hour=5, minute=59, second=59, microsecond=0, tzinfo=timezone.utc)
    # Parameters
    params = {
        "key": api_key,
        "sport": 'Soccer (Football)',
        "league": league,
        "start_date_after": start_date,
        "start_date_before": end_date_cst.isoformat()
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check response
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None

    data = response.json()
    game_ids = [game['id'] for game in data['data']]
    return game_ids


"""

'\ndef get_soccer_game_ids(league):\n    # Endpoint\n    endpoint = "https://api-external.oddsjam.com/api/v2/games"\n\n    # Current week date range\n    today = datetime.now(timezone.utc)\n    start_date = today.isoformat()\n\n\n    # Calculate the upcoming Monday or today if it\'s Monday\n    days_until_monday = (7 - today.weekday()) % 7\n    monday = today + timedelta(days=days_until_monday)\n    \n    # Set time to 11:59 pm and adjust for CST (UTC-6)\n    end_date_cst = (monday + timedelta(days=1)).replace(hour=5, minute=59, second=59, microsecond=0, tzinfo=timezone.utc)\n    # Parameters\n    params = {\n        "key": api_key,\n        "sport": \'Soccer (Football)\',\n        "league": league,\n        "start_date_after": start_date,\n        "start_date_before": end_date_cst.isoformat()\n    }\n\n    # Make the request\n    response = requests.get(endpoint, params=params)\n\n    # Check response\n    if response.status_code != 200:\n        print(f"Error: {response.status_code}"

In [218]:
"""
leagues = ['Germany - Bundesliga', 'Italy - Serie A', 'Spain - La Liga']

caesars_soccer_final = pd.DataFrame()

for league in leagues:
    soccer_ids = get_soccer_game_ids(league)
    soccer_markets = fetch_sports_markets('Soccer', league, api_key, sportsbook=['Pinnacle'])
    soccer_df = fetch_game_data(soccer_ids, soccer_markets, ['Pinnacle', 'BetMGM', 'Caesars'])
    soccer_df_ev = find_plus_ev_bets(soccer_df, thresh=5)
    caesars_soccer = soccer_df_ev[['Game ID', 'Bet Name', "Game Name", 'Market Name', 'line_caesars', 'Odds_caesars', 'line_pinnacle', 'Odds_pinnacle', 'EV_caesars']]
    caesars_soccer = caesars_soccer.sort_values('EV_caesars', ascending=False)
    caesars_soccer_final = caesars_soccer_final.append(caesars_soccer, ignore_index=True)

"""

'\nleagues = [\'Germany - Bundesliga\', \'Italy - Serie A\', \'Spain - La Liga\']\n\ncaesars_soccer_final = pd.DataFrame()\n\nfor league in leagues:\n    soccer_ids = get_soccer_game_ids(league)\n    soccer_markets = fetch_sports_markets(\'Soccer\', league, api_key, sportsbook=[\'Pinnacle\'])\n    soccer_df = fetch_game_data(soccer_ids, soccer_markets, [\'Pinnacle\', \'BetMGM\', \'Caesars\'])\n    soccer_df_ev = find_plus_ev_bets(soccer_df, thresh=5)\n    caesars_soccer = soccer_df_ev[[\'Game ID\', \'Bet Name\', "Game Name", \'Market Name\', \'line_caesars\', \'Odds_caesars\', \'line_pinnacle\', \'Odds_pinnacle\', \'EV_caesars\']]\n    caesars_soccer = caesars_soccer.sort_values(\'EV_caesars\', ascending=False)\n    caesars_soccer_final = caesars_soccer_final.append(caesars_soccer, ignore_index=True)\n\n'

### NFL

In [229]:
## NFL
def get_nfl_game_ids():
    # Endpoint
    endpoint = "https://api-external.oddsjam.com/api/v2/games"

    # Current week date range
    today = datetime.now(timezone.utc)
    start_date = today.isoformat()


    # Calculate the upcoming Monday or today if it's Monday
    days_until_monday = (7 - today.weekday()) % 7
    monday = today + timedelta(days=days_until_monday)
    
    # Set time to 11:59 pm and adjust for CST (UTC-6)
    end_date_cst = (monday + timedelta(days=1)).replace(hour=5, minute=59, second=59, microsecond=0, tzinfo=timezone.utc)
    # Parameters
    params = {
        "key": api_key,
        "sport": "football",
        "league": "NFL",
        "start_date_after": start_date,
        "start_date_before": end_date_cst.isoformat()
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check response
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return None

    data = response.json()
    game_ids = [game['id'] for game in data['data']]
    return game_ids


In [230]:
nfl_markets= fetch_sports_markets('Football', 'NFL', api_key, sportsbook=['Pinnacle'])['name'].tolist()
nfl_player_markets = [market for market in nfl_markets if 'Player' in market]
nfl_game_markets = [market for market in nfl_markets if 'Player' not in market]

nfl_ids = get_nfl_game_ids()

nfl_gdf = fetch_game_data(nfl_ids,nfl_game_markets,['Pinnacle','BetMGM','Caesars'])
nfl_pdf = fetch_game_data(nfl_ids,nfl_player_markets,['Pinnacle','BetMGM','Caesars'])
nfl_gdf['League'] = 'NFL'
nfl_pdf['League'] = 'NFL'

nfl_gdf_ev = find_plus_ev_bets(nfl_gdf,thresh=5)
nfl_pdf_ev = find_plus_ev_bets(nfl_pdf,thresh=5)

nfl_gdf_ev = nfl_gdf_ev[['Game ID','League','Bet Name',"Game Name",'Market Name','line_caesars','Odds_caesars','line_pinnacle','Odds_pinnacle','EV_caesars','true_prob_pinnacle']]
nfl_pdf_ev = nfl_pdf_ev[['Game ID','League','Bet Name',"Game Name",'Market Name','line_caesars','Odds_caesars','line_pinnacle','Odds_pinnacle','EV_caesars','true_prob_pinnacle']]

nfl_ev = pd.concat([nfl_gdf_ev,nfl_pdf_ev])
nfl_ev = nfl_ev[(abs(nfl_ev['Odds_caesars']) > 100) & (abs(nfl_ev['Odds_caesars']) < 1000)]
nfl_ev = nfl_ev[nfl_ev.EV_caesars > 5]

caesars_master = pd.concat([nfl_gdf_ev,nfl_pdf_ev,caesars_ncaa,caesarsdf_mlb_player,caesars_gdf])

caesars_filt = caesars_master[(abs(caesars_master['Odds_caesars']) > 100) & (abs(caesars_master['Odds_caesars']) < 1000)]


caesars_filtered = caesars_filt[caesars_filt.EV_caesars > 5]

caesars_final_notification = caesars_filtered.sort_values('EV_caesars',ascending=False)

def calculate_kelly_bets(df, bankroll=500, daily_bet_limit=0.5):
    # Read the CSV data
    
    # 50% of the bankroll limit
    total_bankroll_limit = bankroll * daily_bet_limit  # 50% of $500
    
    # Define Kelly Criterion
    def kelly_criterion(odds, prob):
        q = 1 - prob
        b = odds / 100 if odds > 0 else abs(100 / odds)  # Converting American odds to decimal
        return (b * prob - q) / b

    # Apply Kelly Criterion to calculate recommended bet sizes
    df['Kelly Fraction'] = df.apply(lambda row: kelly_criterion(row['Odds_caesars'], row['true_prob_pinnacle']), axis=1)
    
    # Calculate bet amounts (multiply Kelly fraction by bankroll)
    df['Recommended Bet Size'] = df['Kelly Fraction'] * bankroll

    # Ensure the total bet amount doesn't exceed 50% of the bankroll
    total_recommended_bet = df['Recommended Bet Size'].sum()
    if total_recommended_bet > total_bankroll_limit:
        scaling_factor = total_bankroll_limit / total_recommended_bet
        df['Recommended Bet Size'] *= scaling_factor
    
    # Return dataframe with recommended bet sizes
    return df[['Game ID','Game Name','Bet Name','Market Name', 'Odds_caesars', 'true_prob_pinnacle', 'Kelly Fraction', 'Recommended Bet Size']]

# Use the function to calculate bet sizes
df_with_recommended_bets = calculate_kelly_bets(caesars_final_notification)


df_with_recommended_bets.to_csv(f'ev_df_{date.today()}',index=False,header=True)